> IDRID (Using the Original Train Test Split Version)

> CNN512 With Dropout - Raw Data

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [23]:
# Load the dataset
data = pd.read_csv("test_split_image_key.csv")

# Split the dataset into training and testing sets
train_data = data[data["split_type"] == "training"]
test_data = data[data["split_type"] == "testing"]

In [24]:
retinopathy_grade_dtype = data["Retinopathy grade"].dtype
print("Retinopathy Grade Data Type:", retinopathy_grade_dtype)

Retinopathy Grade Data Type: int64


In [25]:
# Function to load images as they are, without preprocessing
def load_image(image_path):
    # Load and resize the image to 512x512
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (512, 512))
    return resized_image


# Load training and testing images into numpy arrays
X_train = [
    load_image("a_training/" + img_name + ".jpg")
    for img_name in train_data["Image name"]
]
X_test = [
    load_image("b_testing/" + img_name + ".jpg") for img_name in test_data["Image name"]
]

X_train = np.array(X_train)
X_test = np.array(X_test)

# Extract labels directly without one-hot encoding
y_train = train_data["Retinopathy grade"].values
y_test = test_data["Retinopathy grade"].values

In [26]:
input_shape = (512, 512, 3)
num_classes = 5
epochs = 10  # placeholder

In [9]:
cnn512_dropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [10]:
cnn512_dropout.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d (ZeroPadding2D)  │ (None, 516, 516, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 514, 514, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 514, 514, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 514, 514, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 257, 257, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 255, 255, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 255, 255, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 255, 255, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 125, 125, 96)   │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 125, 125, 96)   │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 125, 125, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 60, 60, 96)     │        83,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 60, 60, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 60, 60, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 128)    │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 200)    │       230,600 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 8,211,613 (31.32 MB)

 Trainable params: 8,207,381 (31.31 MB)

 Non-trainable params: 4,232 (16.53 KB)

In [11]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [12]:
# fitting the model
history_cnn512_dropout = cnn512_dropout.fit(
    X_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 138s 10s/step - accuracy: 0.3048 - loss: 2.1785 - learning_rate: 1.0000e-04
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 159s 12s/step - accuracy: 0.2594 - loss: 2.2079 - learning_rate: 0.0101
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 177s 14s/step - accuracy: 0.3280 - loss: 1.9119 - learning_rate: 0.0201
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 179s 14s/step - accuracy: 0.2948 - loss: 1.9314 - learning_rate: 0.0301
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 182s 14s/step - accuracy: 0.2827 - loss: 2.1167 - learning_rate: 0.0401
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 186s 13s/step - accuracy: 0.2450 - loss: 2.4430 - learning_rate: 0.0501
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 200s 15s/step - accuracy: 0.2700 - loss: 3.4005 - learning_rate: 0.0600
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 158s 12s/step - accuracy: 0.2474 - loss: 3.2149 - learning_rate: 0.0700
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 200s 16s/step - accuracy: 0.2711 - loss: 2.7589 - learning_rate: 0.0800
Epoch 

In [13]:
# Evaluate the model
test_loss, test_accuracy = cnn512_dropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 828ms/step - accuracy: 0.2570 - loss: 1305.6583
Test Accuracy: 0.3300970792770386


> CNN512 With Dropout - Normalize by 225

In [27]:
X_train = X_train.astype("float32") / 225  # normalize
X_test = X_test.astype("float32") / 225  # normalize

In [28]:
cnn512_dropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [29]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [30]:
# fitting the model
history_cnn512_dropout = cnn512_dropout.fit(
    X_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 185s 14s/step - accuracy: 0.1837 - loss: 2.5687 - learning_rate: 1.0000e-04
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 172s 13s/step - accuracy: 0.2528 - loss: 2.0654 - learning_rate: 0.0101
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 158s 11s/step - accuracy: 0.2700 - loss: 1.8377 - learning_rate: 0.0201
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 164s 13s/step - accuracy: 0.2891 - loss: 1.8358 - learning_rate: 0.0301
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 177s 13s/step - accuracy: 0.2649 - loss: 2.2102 - learning_rate: 0.0401
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 186s 14s/step - accuracy: 0.2601 - loss: 2.5273 - learning_rate: 0.0501
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 208s 16s/step - accuracy: 0.2141 - loss: 3.6990 - learning_rate: 0.0600
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 183s 14s/step - accuracy: 0.2607 - loss: 3.1778 - learning_rate: 0.0700
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 220s 17s/step - accuracy: 0.1966 - loss: 3.0390 - learning_rate: 0.0800
Epoch 

In [31]:
# Evaluate the model
test_loss, test_accuracy = cnn512_dropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.3149 - loss: 14765.3281
Test Accuracy: 0.3106796145439148


> CNN512 With Dropout - Normalize by 225 + Augmentation 20 times

In [15]:
cnn512_dropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [16]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# Define the data augmentation generator
data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Random rotation between 0 and 20 degrees
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode="nearest",  # Strategy for filling in newly created pixels
)

# Apply the data augmentation using .flow() and then fit the model
augmented_data = data_augmentation.flow(X_train, y_train, batch_size=32)

In [19]:
import math

# Fit the model with the augmented data generator
history_cnn512_dropout = cnn512_dropout.fit(
    augmented_data,
    steps_per_epoch=math.ceil(len(X_train) / 32),  # Round up to ensure it's an integer
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Epoch 1/10


/Users/tinayiluo/miniconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 182s 13s/step - accuracy: 0.1481 - loss: 2.2858 - learning_rate: 1.0000e-04
Epoch 2/10


2024-04-02 16:48:48.805321: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0101
Epoch 3/10


...

2024-04-02 16:55:59.173839: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


13/13 ━━━━━━━━━━━━━━━━━━━━ 251s 19s/step - accuracy: 0.2145 - loss: 2.9734 - learning_rate: 0.0600
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0700
Epoch 9/10


2024-04-02 17:00:10.828761: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


13/13 ━━━━━━━━━━━━━━━━━━━━ 191s 14s/step - accuracy: 0.2448 - loss: 2.9026 - learning_rate: 0.0800
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0900


2024-04-02 17:03:22.221637: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [20]:
# Evaluate the model
test_loss, test_accuracy = cnn512_dropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.2446 - loss: 2541.9651
Test Accuracy: 0.18446601927280426
